In [7]:
# Useful starting lines
%matplotlib inline
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read data

In [8]:
from sqlalchemy import create_engine
import pandas as pd

path = "../wjazzd.db"
engine = create_engine(f"sqlite:///{path}")

melody = pd.read_sql("melody", engine)
beats = pd.read_sql("beats", engine)

df_melody = pd.DataFrame(melody)
df_beats = pd.DataFrame(beats)

## Fill the chords column with the last known chord

In [9]:
#df_chords = df_beats[df_beats['chord'] != '']
df_chords = df_beats.replace({'chord': {'': np.nan}}).ffill()
#df_chords.to_csv('df_chords.csv')
df_chords.info()
df_chords.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132329 entries, 0 to 132328
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   beatid      132329 non-null  int64  
 1   melid       132329 non-null  int64  
 2   onset       132329 non-null  float64
 3   bar         132329 non-null  int64  
 4   beat        132329 non-null  int64  
 5   signature   132329 non-null  object 
 6   chord       132325 non-null  object 
 7   form        132329 non-null  object 
 8   bass_pitch  132329 non-null  float64
 9   chorus_id   132329 non-null  int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 10.1+ MB


,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
0,1,1,9.171882,-1,1,,NaN,I1,42.0,0
1,2,1,9.488254,-1,2,,NaN,,42.0,0
2,3,1,9.779955,-1,3,,NaN,,40.0,0
3,4,1,10.052608,-1,4,,NaN,,40.0,0
4,5,1,10.339796,0,1,,Bb6,,50.0,0


## Extract corresponding row in beats using the (melid, bar, beat) features

In [10]:
# Define new index with the key (melid, bar, beat)
df_chords_new = df_chords.set_index(['melid', 'bar', 'beat'], drop=False)
df_melody_new = df_melody.set_index(['melid', 'bar', 'beat'], drop=False)

# Concatenate the dataframes using the new index and then reset the index again
df_mel_beats = pd.concat([df_melody_new, df_chords_new.reindex(df_melody_new.index)], axis=1)
df_mel_beats = df_mel_beats.reset_index(drop=True)

# Remove duplicate columns
# Duplicated columns are:
# - melid
# - onset
# - bar
# - beat
df_mel_beats = df_mel_beats.loc[:,~df_mel_beats.columns.duplicated()]

df_mel_beats.to_csv('../Data/df_mel_beats.csv')

df_mel_beats.info()
df_mel_beats.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200809 entries, 0 to 200808
Data columns (total 32 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   eventid      200809 non-null  int64  
 1   melid        200809 non-null  int64  
 2   onset        200809 non-null  float64
 3   pitch        200809 non-null  float64
 4   duration     200809 non-null  float64
 5   period       200809 non-null  int64  
 6   division     200809 non-null  int64  
 7   bar          200809 non-null  int64  
 8   beat         200809 non-null  int64  
 9   tatum        200809 non-null  int64  
 10  subtatum     200809 non-null  int64  
 11  num          200809 non-null  int64  
 12  denom        200809 non-null  int64  
 13  beatprops    200809 non-null  object 
 14  beatdur      200809 non-null  float64
 15  tatumprops   200809 non-null  object 
 16  f0_mod       200809 non-null  object 
 17  loud_max     180195 non-null  float64
 18  loud_med     200320 non-

,eventid,melid,onset,pitch,duration,period,division,bar,beat,tatum,...,loud_s2b,f0_range,f0_freq_hz,f0_med_dev,beatid,signature,chord,form,bass_pitch,chorus_id
0,1,1,10.343492,65.0,0.138776,4,1,0,1,1,...,1.056169,37.794261,12.932532,-0.328442,5,,Bb6,,50.0,0
1,2,1,10.637642,63.0,0.171247,4,4,0,2,1,...,1.120317,6.365930,6.956935,11.135423,6,,Bb6,,50.0,0
2,3,1,10.843719,58.0,0.081270,4,4,0,2,4,...,1.310389,68.010392,NaN,32.366787,6,,Bb6,,50.0,0
3,4,1,10.948209,61.0,0.235102,4,1,0,3,1,...,0.984047,15.443906,5.867151,-3.374696,7,,Bb6,,50.0,0
4,5,1,11.232653,63.0,0.130612,4,1,0,4,1,...,1.061262,11.444363,8.329975,6.377737,8,,Bb6,,42.0,0


In [11]:
import sys
sys.path.append('../')

from combine_melody_beats import combine_melody_beats

df_mel_beats.equals(combine_melody_beats(df_melody, df_beats))

True